In [1]:
import pandas as pd

In [2]:

!unzip '/content/drive/MyDrive/archive (3).zip'
     

Archive:  /content/drive/MyDrive/archive (3).zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_data/blasti/DSC_6410.jpg  
  inflating: test_data/test_data/blasti/DSC_6411.jpg  
  inflating: test_data/test_data/bonegl/DSC_4587.jpg  
  inflating: test_data/test_data/bonegl/DSC_4588.jpg  
  inflating: tes

In [3]:

# Data Augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator
     

In [4]:

train_gen = ImageDataGenerator(rescale=(1./255),horizontal_flip=True,shear_range=0.2)
test_gen = ImageDataGenerator(rescale=(1./255))  #--> (0 to 255) convert to (0 to 1)

In [5]:

train = train_gen.flow_from_directory('/content/train_data/train_data',
                                      target_size=(120, 120),
                                      class_mode='categorical', 
                                      batch_size=8)
test = test_gen.flow_from_directory('/content/test_data/test_data',
                                    target_size=(120, 120),
                                      class_mode='categorical', 
                                      batch_size=8)

Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


In [6]:

train.class_indices

{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

In [7]:

# CNN

from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import BatchNormalization, Dropout
     

In [8]:

# Initializing the seq model
model = Sequential()
# Adding conv layer with input
model.add(Convolution2D(12,(3,3),activation='relu',input_shape=(120, 120, 3)))
# Normalizing the conv layer output
model.add(BatchNormalization())
# Selecting the max values
model.add(MaxPooling2D(pool_size=(2,2)))
# Dropping the unwanted 20% of data
model.add(Dropout(0.2))
model.add(Convolution2D(24,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Convolution2D(36,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
# Hiddern layers
model.add(Dense(62,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
# Output layer
model.add(Dense(16,activation='softmax'))

In [9]:

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 12)      336       
                                                                 
 batch_normalization (BatchN  (None, 118, 118, 12)     48        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 12)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 59, 59, 12)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 24)        2616      
                                                                 
 batch_normalization_1 (Batc  (None, 57, 57, 24)       9

In [10]:

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:

# Early stopping
from tensorflow.keras.callbacks import EarlyStopping
     

In [12]:

early_stop = EarlyStopping(monitor='accuracy',patience=8)

In [14]:
history = model.fit(train,batch_size=4,validation_data=test,epochs=50,callbacks=early_stop)

Epoch 1/50
19/19 [==============================] - 129s 7s/step - loss: 0.6340 - accuracy: 0.8267 - val_loss: 3.8682 - val_accuracy: 0.1083
Epoch 2/50
19/19 [==============================] - 128s 7s/step - loss: 0.6151 - accuracy: 0.8333 - val_loss: 3.3056 - val_accuracy: 0.2229
Epoch 3/50
19/19 [==============================] - 131s 7s/step - loss: 0.7044 - accuracy: 0.8000 - val_loss: 3.1687 - val_accuracy: 0.2739
Epoch 4/50
19/19 [==============================] - 107s 6s/step - loss: 0.6259 - accuracy: 0.8200 - val_loss: 3.4863 - val_accuracy: 0.1592
Epoch 5/50
19/19 [==============================] - 128s 7s/step - loss: 0.5439 - accuracy: 0.8467 - val_loss: 2.9825 - val_accuracy: 0.2229
Epoch 6/50
19/19 [==============================] - 105s 6s/step - loss: 0.5510 - accuracy: 0.8600 - val_loss: 3.3396 - val_accuracy: 0.2484
Epoch 7/50
19/19 [==============================] - 128s 7s/step - loss: 0.4470 - accuracy: 0.8800 - val_loss: 3.2275 - val_accuracy: 0.2739
Epoch 8/50
19

In [15]:

# Testing

import numpy as np
from tensorflow.keras.preprocessing import image
img2 = image.load_img('/content/test_data/test_data/brhkyt/D72_0474.jpg',target_size=(120,120))
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2,axis=0)
pred = np.argmax(model.predict(img2))
print(pred)
#output = ['bears', 'crows', 'elephants', 'rats']
#print(output[pred])

1/1 [==============================] - 0s 220ms/step
8


In [16]:
d  = train.class_indices
dict = {value:key for key, value in d.items()}
print(dict[pred])

himgri
